# Lab 4 - Le Nettoyage de Données avec Pandas

## Objectifs d'apprentissage

A la fin de ce laboratoire, vous saurez :
1. Identifier les problemes de qualite dans un jeu de donnees (valeurs manquantes, types incorrects)
2. Appliquer des strategies de nettoyage appropriees (dropna, fillna, imputation intelligente)
3. Transformer et enrichir les donnees (conversion de types, creation de colonnes)
4. Effectuer des agregations pour repondre a des questions metier

### Prerequis
- Python 3.10+
- Connaissance de base de Pandas
- Fichier transactions.csv dans le repertoire courant

### Duree estimee : 30-45 minutes

## Introduction : La Règle des 80/20 en Data Science

En Data Science, on dit souvent que 80% du temps est consacré à la préparation et au nettoyage des données, et seulement 20% à l'analyse et à la modélisation. Cette étape, bien que moins glamour, est absolument cruciale. Des données de mauvaise qualité mènent inévitablement à des modèles et des conclusions de mauvaise qualité ('Garbage In, Garbage Out').

Dans ce laboratoire, nous allons aborder de front ce travail de préparation en utilisant un jeu de données volontairement "sale".

### Étape 1 : Inspection des Données

In [1]:
import pandas as pd

# Charger le jeu de données
df = pd.read_csv('transactions.csv')

# Afficher les premières lignes
df.head()

,date,id_produit,categorie,quantite,prix_unitaire
0,2023-10-01,A101,Electronique,5.0,120.0
1,2023-10-01,B202,Livre,10.0,15.5
2,2023-10-02,A101,Electronique,3.0,NaN
3,2023-10-03,C303,Maison,2.0,75.9
4,2023-10-03,B202,Livre,6.0,15.5


### Vue d'ensemble de la structure

Apres avoir examine les premieres lignes, explorons la structure globale du DataFrame : types de colonnes, nombre de valeurs non-nulles, et memoire utilisee. Cette etape nous revelera les problemes potentiels (types incorrects, valeurs manquantes).

In [2]:
# Obtenir un résumé des informations du DataFrame
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           7 non-null      str    
 1   id_produit     7 non-null      str    
 2   categorie      7 non-null      str    
 3   quantite       6 non-null      float64
 4   prix_unitaire  6 non-null      float64
dtypes: float64(2), str(3)
memory usage: 570.0 bytes


### Analyse des valeurs manquantes

Maintenant, comptabilisons precisement les valeurs manquantes par colonne. Cela nous permettra de prioriser les actions de nettoyage et de choisir la strategie appropriee (suppression, imputation, etc.).

In [3]:
# Compter les valeurs manquantes par colonne
df.isnull().sum()

date             0
id_produit       0
categorie        0
quantite         1
prix_unitaire    1
dtype: int64

**Observations :**
- La colonne `date` est de type `object` (chaîne de caractères) alors qu'elle devrait être de type `datetime`.
- Il y a une valeur manquante dans `quantite`.
- Il y a une valeur manquante dans `prix_unitaire`.

### Étape 2 : Nettoyage - Gestion des Valeurs Manquantes

In [4]:
# Supprimer les lignes où la quantité est manquante
# Une transaction sans quantité n'est pas exploitable
df.dropna(subset=['quantite'], inplace=True)

### Imputation intelligente des prix manquants

Pour les prix unitaires manquants, nous allons utiliser une strategie d'imputation plus sophistiquee que la simple moyenne globale : nous remplacerons chaque valeur manquante par la moyenne des prix du meme produit. Cette approche preserve mieux la coherence economique des donnees.

In [5]:
# Remplir le prix manquant avec la moyenne des prix du même produit
# C'est une stratégie d'imputation plus intelligente que la moyenne globale
df['prix_unitaire'] = df.groupby('id_produit')['prix_unitaire'].transform(lambda x: x.fillna(x.mean()))

# Vérifions s'il reste des valeurs nulles
df.isnull().sum()

date             0
id_produit       0
categorie        0
quantite         0
prix_unitaire    0
dtype: int64

### Étape 3 : Correction des Types

In [6]:
# Convertir la colonne 'date' en datetime
# errors='coerce' transformera les dates invalides en NaT (Not a Time)
df['date'] = pd.to_datetime(df['date'], errors='coerce')

### Verification de la conversion

Verifions que la colonne `date` a bien ete convertie en type datetime et que les autres colonnes sont correctement typees. Cela nous permettra d'exploiter les fonctionnalites temporelles de Pandas (extraction de mois, annee, tri chronologique, etc.).

In [7]:
# Vérifions les types à nouveau
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           5 non-null      datetime64[us]
 1   id_produit     6 non-null      str           
 2   categorie      6 non-null      str           
 3   quantite       6 non-null      float64       
 4   prix_unitaire  6 non-null      float64       
dtypes: datetime64[us](1), float64(2), str(2)
memory usage: 450.0 bytes


### Étape 4 : Transformation - Création de Colonnes

In [8]:
# Créer une colonne 'chiffre_affaires'
df['chiffre_affaires'] = df['quantite'] * df['prix_unitaire']
df.head()

,date,id_produit,categorie,quantite,prix_unitaire,chiffre_affaires
0,2023-10-01,A101,Electronique,5.0,120.0,600.0
1,2023-10-01,B202,Livre,10.0,15.5,155.0
2,2023-10-02,A101,Electronique,3.0,120.0,360.0
3,2023-10-03,C303,Maison,2.0,75.9,151.8
4,2023-10-03,B202,Livre,6.0,15.5,93.0


### Étape 5 : Agrégation pour l'Analyse

In [9]:
# Question métier : Quel est le chiffre d'affaires total par catégorie de produit ?
ca_par_categorie = df.groupby('categorie')['chiffre_affaires'].sum().reset_index()

print("Chiffre d'affaires total par catégorie :")
print(ca_par_categorie)

Chiffre d'affaires total par catégorie :
      categorie  chiffre_affaires
0  Electronique            1440.0
1         Livre             248.0
2        Maison             151.8


## Conclusion

Félicitations ! Vous avez nettoyé un jeu de données, corrigé ses types, créé de nouvelles informations et répondu à une question métier grâce à une agrégation.

Vous avez appris à :
- Identifier les problèmes avec `.info()` et `.isnull()`.
- Gérer les valeurs manquantes avec `dropna` et `fillna` (de manière avancée avec `groupby` et `transform`).
- Corriger les types de données avec `pd.to_datetime`.
- Créer de nouvelles colonnes pour enrichir vos données.
- Agréger les données avec `groupby()` pour obtenir des insights.

**Lien avec l'après-midi :** Ces étapes de nettoyage et d'analyse sont exactement le type de tâches qu'un agent d'IA pourra bientôt réaliser de manière autonome. En maîtrisant ces techniques, vous comprenez la logique qu'il faudra implémenter dans les "outils" que nous fournirons à nos agents pour qu'ils puissent travailler efficacement avec des données du monde réel.